In [1]:
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


In [3]:
def data_add(soup):
    #設定初始值
    try:
        data = {}
        res = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')
        
        for item in res:
            data.setdefault(count,{})
            #分類或來源
            data[count]['category'] = item.find('div',{'class':'sc-gacfCG bPSpUf'}).string
            #文章來源網址
            data[count]['link'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
            #標題
            data[count]['title'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
            #簡介
            data[count]['summary'] = item.find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).string
            
            #來源網址(若非聯外網站則自動輸入為cupoy網頁)
            source_data = re.findall('[\S]+com',item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href'))
            if len(source_data)>0:
                data[count]['source'] = source_data[0]
            else:
                data[count]['source'] = 'https://www.cupoy.com'
            
            count +=1

    except:
        print('Adding Fail')
        pass
              
    return data
    

In [8]:
aaaa = {}
count = 1
browser = webdriver.Chrome('../chromedriver.exe')
url = 'https://www.cupoy.com/newsfeed/topstory'
browser.get(url)
time.sleep(10)
while True:
    
    # get infos
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    
    # update infos (remove duplicate information!!)
    bbbb = data_add(soup)
    print(bbbb)
    print(f"update... {count}")
    aaaa.update(bbbb)
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the page.
    time.sleep(1)

    # Calculate new scroll height and compare with last scroll height.
    browser.execute_script("return document.body.scrollHeight")
    
    
    break
'''    
    if count > 30:
        print("collected over")
        break
    time.sleep(5) 
'''
browser.quit()
aaaa

{1: {'category': 'The News Lens 關鍵評論網', 'link': 'https://www.thenewslens.com/article/133944', 'title': 'Google趨勢：哪國「洗手」搜尋量較多？和後續的疫情控制有關係嗎？ - The News Lens 關鍵評論網', 'summary': '臺灣的防疫政策確實提升了民眾的健康識能，顯著提升臺灣民眾洗手的健康意識。而且臺灣民眾重視洗手的程度，是疫情大爆發時領先世界各國，卻不易察覺的保護因子，應該要繼續保持。', 'source': 'https://www.thenewslens.com'}}
update... 1


{1: {'category': 'The News Lens 關鍵評論網',
  'link': 'https://www.thenewslens.com/article/133944',
  'title': 'Google趨勢：哪國「洗手」搜尋量較多？和後續的疫情控制有關係嗎？ - The News Lens 關鍵評論網',
  'summary': '臺灣的防疫政策確實提升了民眾的健康識能，顯著提升臺灣民眾洗手的健康意識。而且臺灣民眾重視洗手的程度，是疫情大爆發時領先世界各國，卻不易察覺的保護因子，應該要繼續保持。',
  'source': 'https://www.thenewslens.com'}}

In [53]:
browser = webdriver.Chrome('../chromedriver.exe')

#setting initial parameter 
url = 'https://www.cupoy.com/newsfeed/topstory'
#res_num = 0  #一個起始數字
switch = True
count = 1
data = {}
#執行讀取，等待回應
browser.get(url)
time.sleep(10)



#用來源網址做感測器勘誤

#設定文章數偵測器
while switch :
    soup = BeautifulSoup(browser.page_source,'html.parser')
    res = soup.find_all('div', class_='sc-eEieub sc-iuDHTM ibJqYc')
    
    #用迴圈處理資料
    for item in res:
        data.setdefault(count,{})
        det = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
        if det not in data:
            #分類或來源
            data[count]['category'] = item.find('div',{'class':'sc-gacfCG bPSpUf'}).string
            #文章來源網址
            data[count]['link'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
            #標題
            data[count]['title'] = item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
            #簡介
            data[count]['summary'] = item.find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).string

            #來源網址(若非聯外網站則自動輸入為cupoy網頁)
            source_data = re.findall('[\S]+com',item.find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href'))
            if len(source_data)>0:
                data[count]['source'] = source_data[0]
            else:
                data[count]['source'] = 'https://www.cupoy.com'
        
            count +=1
        else:
            continue


    #設定預定要爬取的文章數量並用回存數量作為比對停止器
    if count >= 500:
        switch = False
    else:
        continue
    
    
    #模擬器下拉畫面
    time.sleep(5)
    browser.execute_script("window.scrollBy(0,10000)")    

#關閉虛擬瀏覽器
browser.quit()
df = pd.DataFrame(data)
df = df.T
df.head(500)

,category,link,title,summary,source
1,科技,https://technews.tw/2020/04/16/tsmc-outlook-at...,臺積電維持原有資本支出不變，3 奈米製程預計 2021 年試產 | TechNews 科技新報,"晶圓代工龍頭臺積電 16 日發表 2020 年第 1 季財報，營收金額達到新臺幣 3,106...",https://www.cupoy.com
2,科技,https://technews.tw/2020/04/17/infineon-succes...,英飛凌宣佈完成 87 億美元收購賽普拉斯，將一躍成車用半導體龍頭 | TechNews 科技新報,德國科技大廠英飛凌（Infineon Technologies）宣佈，已經完成收購賽普拉斯半...,https://www.cupoy.com
3,科技,https://technews.tw/2020/04/17/huawei-chip-uni...,憂臺積電受美限制，傳華為晶片擴大轉單中芯 | TechNews 科技新報,路透社 16 日引述數名知情人士報導，中國電信設備巨擘華為因應美國可能祭出更多封殺措施，正逐...,https://www.cupoy.com
4,科技,https://technews.tw/2020/04/17/unpaid-leave-su...,全臺無薪假激增破萬，竹科也不例外 | TechNews 科技新報,由於受到武漢肺炎疫情衝擊，無薪假人數正持續增加，據勞動部統計，全臺已破萬人。\r\r\r\r...,https://www.cupoy.com
5,職場白領,http://sc.piee.pw/HDBAH,「巧妙的」的英文怎麼說？ | 經理人,ingenious 可以用來形容一個人富有智慧、謀略，相似詞有 clever（聰明）、cun...,https://www.cupoy.com
...,...,...,...,...,...
496,科技,https://technews.tw/2020/04/16/tsmc-outlook-at...,臺積電維持原有資本支出不變，3 奈米製程預計 2021 年試產 | TechNews 科技新報,"晶圓代工龍頭臺積電 16 日發表 2020 年第 1 季財報，營收金額達到新臺幣 3,106...",https://www.cupoy.com
497,科技,https://technews.tw/2020/04/17/infineon-succes...,英飛凌宣佈完成 87 億美元收購賽普拉斯，將一躍成車用半導體龍頭 | TechNews 科技新報,德國科技大廠英飛凌（Infineon Technologies）宣佈，已經完成收購賽普拉斯半...,https://www.cupoy.com
498,科技,https://technews.tw/2020/04/17/huawei-chip-uni...,憂臺積電受美限制，傳華為晶片擴大轉單中芯 | TechNews 科技新報,路透社 16 日引述數名知情人士報導，中國電信設備巨擘華為因應美國可能祭出更多封殺措施，正逐...,https://www.cupoy.com
499,科技,https://technews.tw/2020/04/17/unpaid-leave-su...,全臺無薪假激增破萬，竹科也不例外 | TechNews 科技新報,由於受到武漢肺炎疫情衝擊，無薪假人數正持續增加，據勞動部統計，全臺已破萬人。\r\r\r\r...,https://www.cupoy.com


In [54]:
#測試


soup = BeautifulSoup(browser.page_source,'html.parser')
res = soup.find_all('div',{'class':'sc-eEieub sc-iuDHTM ibJqYc'})
#分類或來源
a = res[0].find('div',{'class':'sc-gacfCG bPSpUf'}).string
#文章來源網址
b = res[0].find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('href')
#標題
c = res[0].find('a',{'class':'sc-RbTVP sc-kEmuub eSJXii'}).get('title')
#簡介
d = res[0].find('p',{'class':'sc-FQuPU sc-ciodno bvnzOw'}).text
#來源網址
e = re.findall('[\S]+com',b)

e

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63796): Max retries exceeded with url: /session/36eb241bd4d189b133325d46307a5228/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022E984FEF08>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。'))